In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [2]:
unp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",",")\
   .csv("file:////home/abhijeet/spark coding/codes.csv")

unp.printSchema()
unp.registerTempTable('u')
unp.show()

root
 |-- Document_Number: long (nullable = true)
 |-- keydate_prompt: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- oib: long (nullable = true)
 |-- collection_level: integer (nullable = true)
 |-- Dunning_Date: integer (nullable = true)
 |-- Dunning_Amount: double (nullable = true)

+---------------+--------------+------+-----------+----------------+------------+--------------+
|Document_Number|keydate_prompt|amount|        oib|collection_level|Dunning_Date|Dunning_Amount|
+---------------+--------------+------+-----------+----------------+------------+--------------+
|    19153639679|      20190831|380.33|57240799650|              60|    20190729|        380.33|
|    19153639679|      20191130|380.33|57240799650|              80|    20191030|        380.33|
|    19153639679|      20191130|380.33|57240799650|              70|    20190925|        380.33|
|    19153639679|      20191130|380.33|57240799650|              63|    20190924|        760.66|
|    1915363

In [3]:

unp.withColumn("collection_level", f.expr("CAST(collection_level AS string)"))
unp.registerTempTable('u')

In [4]:
df1=unp.withColumn("Interval",f.expr("case when collection_level>=10 and collection_level<=19 then 1019\
                                        when collection_level>=20 and collection_level<=29 then 2029\
                                        when collection_level>=30 and collection_level<=39 then 3039\
                                        when collection_level>=40 and collection_level<=49 then 4049\
                                        when collection_level>=50 and collection_level<=79 then '50_79'\
                                        when collection_level>=80 and collection_level<=94 then 8094\
                                        when collection_level=95 then 95\
                                        when collection_level=0 then 'No_DUN' end"))

df1.show()
df1.registerTempTable('v')

+---------------+--------------+------+-----------+----------------+------------+--------------+--------+
|Document_Number|keydate_prompt|amount|        oib|collection_level|Dunning_Date|Dunning_Amount|Interval|
+---------------+--------------+------+-----------+----------------+------------+--------------+--------+
|    19153639679|      20190831|380.33|57240799650|              60|    20190729|        380.33|   50_79|
|    19153639679|      20191130|380.33|57240799650|              80|    20191030|        380.33|    8094|
|    19153639679|      20191130|380.33|57240799650|              70|    20190925|        380.33|   50_79|
|    19153639679|      20191130|380.33|57240799650|              63|    20190924|        760.66|   50_79|
|    19153639679|      20191130|380.33|57240799650|              60|    20190729|        380.33|   50_79|
|    19153639679|      20191031|380.33|57240799650|              80|    20191030|        380.33|    8094|
|    19153639679|      20191031|380.33|5724079

In [8]:
df2=spark.sql("select v.Document_Number,v.keydate_prompt,v.oib,v.amount,max(v.Dunning_Date) from v \
group by v.Document_Number,v.keydate_prompt,v.oib,v.amount ")
df2.show()

+---------------+--------------+-----------+------+-----------------+
|Document_Number|keydate_prompt|        oib|amount|max(Dunning_Date)|
+---------------+--------------+-----------+------+-----------------+
|    19153639679|      20190731|57240799650|380.33|         20190729|
|    19153639679|      20191130|57240799650|380.33|         20191030|
|    19153909359|      20200531|15200435096|137.75|         20200508|
|    19153639679|      20190831|57240799650|380.33|         20190729|
|    19153639679|      20191031|57240799650|380.33|         20191030|
|    19153909359|      20200430|15200435096|137.75|         20200427|
|    19153639679|      20190930|57240799650|380.33|         20190925|
+---------------+--------------+-----------+------+-----------------+



In [ ]:
df3=spark.sql("c from v \
group by v.keydate_prompt,v.oib,v.amount,v.Level")
df3.show()

In [ ]:
df2=df1.groupBy("Document_Number","keydate_prompt","amount","oib","collection_level",f.max("Dunning_Date"),"Dunning_Amount","Level")\
.agg('Dunning_Date')
df2.show()

In [20]:
df3=spark.sql("select v.Document_Number,v.keydate_prompt,v.oib,v.amount,v.Dunning_Date,v.Dunning_Amount,\
v.collection_level,v.Interval from v ")
df3.show()

+---------------+--------------+-----------+------+------------+--------------+----------------+--------+
|Document_Number|keydate_prompt|        oib|amount|Dunning_Date|Dunning_Amount|collection_level|Interval|
+---------------+--------------+-----------+------+------------+--------------+----------------+--------+
|    19153639679|      20190831|57240799650|380.33|    20190729|        380.33|              60|    5079|
|    19153639679|      20191130|57240799650|380.33|    20191030|        380.33|              80|    8094|
|    19153639679|      20191130|57240799650|380.33|    20190925|        380.33|              70|    5079|
|    19153639679|      20191130|57240799650|380.33|    20190924|        760.66|              63|    5079|
|    19153639679|      20191130|57240799650|380.33|    20190729|        380.33|              60|    5079|
|    19153639679|      20191031|57240799650|380.33|    20191030|        380.33|              80|    8094|
|    19153639679|      20191031|57240799650|38